С каждой страницы, описывающей предложение по аренде конкретного автомобиля у конкретного владельца была собрана следующая информация:

- Параметры аренды:
 - Минимальнвя цена аренды в день (Price)
 - Скидки в зависимости от продолжительности аренды (Sale1, Sale2, Sale3, Sale4)
 - Количество фотографий автомобиля (PhotosCar)
 - Рейтинг предложения (Rating)
 - Количество отзывов (N_feedback)
 - Город (City)
 - Метро (Metro)
 - Сумма депозита (Deposit)
 - Включенные КМ (KmIncluded)
 - Каско (Kasko)
 - Осаго (Osago)

- Характеристики автомобиля:
 - Марка автомобиля (Brand)
 - Модель автомобиля (Model)
 - Описание автомобиля (Description)
 - Вид двигателя (Engine)
 - Марка топлива (Fuel)
 - Объем двигателя (Volume)
 - Вид коробки передач (Gearbox)
 - Вид кузова (Body)
 - Вид привода (Drive)
 - Экологический стандарт ЕС (EcologyEU)
 - Экологический стандарт РФ (EcologyRF)
 - Сторона руля (Rudder)
 - Количество дверей (Doors)
 - Пробег (Run)
 - Вид салона
 - Количество мест (Capacity)
 - Год выпуска (Year)
 - Класс автомобиля (Class)

- Дополнительные услуги:
 - Без депозита (NoDeposit)
 - Доставка (Delivery)
 - Возврат (ReturnAbility)
 - Без ограничения пробега (KmUnlimited)
 - Детское кресло (ChildSeat)
 - Багажник на крышу (RoofRack)

- Характеристики владельца:
 - Средняя оценка арендаторов (RatingOwner)
 - Процент рекомендаций от арендаторов (LoyaltyOwner)
 - Количество автомобилей в каталоге (CarsOwner)
 - Средняя стоимость автомобилей (AvgPriceOwner)
 - Имя владельца (Owner)
 - Наличие фото владельца (PhotoOwner)

Дополнительно собраны:
- URL для каждого наблюдения (URL)
- Список характеристик автомобиля (CharacterCar)
- Список характеристик владельца (CharacterOwner)
- Список дополнительных услуг (Services)

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from fake_useragent import UserAgent
from time import time
from tqdm import tqdm

In [3]:
pd.set_option('display.max_columns', None)

In [15]:
def get_car_data(treeCar, car):
    
    #блоки данных со страницы
    characteristics = treeCar.find_all('div', {'class': 'val'})
    characteristics_names = treeCar.find_all('div', {'class': 'name'})
    characteristics_owner = treeCar.find_all('div', {'class': 'value'})
    characteristics_owner_names = treeCar.find_all('div', {'class': 'label'})
    services = treeCar.find_all('div', {'class': 'title'})
    prices = treeCar.find_all('div', {'class':'car-prices-list--item'})
    character = treeCar.find_all('a', {'class': 'no-decoration'})
    
    #обработанные списки данных
    char_values = list(map(lambda x: x.text, characteristics))
    char_names = list(map(lambda x: x.text, characteristics_names))
    char_owner = list(map(lambda x: x.text.strip(), characteristics_owner))
    char_owner_names = list(map(lambda x: x.text.strip(), characteristics_owner_names))
    services = list(map(lambda x: x.text.strip(), services))
    
    #готовые переменные
    photos_car=int(treeCar.find('div', {'class':'button'}).text[:-4].strip())
    price_day=treeCar.find('div', {'class':'price-per-day'}).text[4:-10].strip()
    car_class = character[1].text.strip()
    brand = character[2].text.strip()
    model = character[3].text.strip()
    owner = char_names[-1]
    rating = car.find('div', {'class': 'rating-value'}).text
    n_feedback = car.find('div', {'class': 'car-review-count'}).text.strip()
    photo_owner = treeCar.find('div', {'class': 'img'}).get('lazy-background')
    
    #цены и скидки
    try:
        sale1 = prices[0].text
    except:
        sale1 = None
    try:
        sale2 = prices[1].text
    except:
        sale2 = None
    try:
        sale3 = prices[2].text
    except:
        sale3 = None
    try:
        sale4 = prices[3].text
    except:
        sale4 = None
    
    #быстрое бронирование
    quick = car.find('div', {'class': 'quick-rent'})
    if quick != None:
        quick = 1
    
    #создаем переменные
    gearbox=engine=body=drive=ecologyEU=ecologyRF=rudder=doors=run=salon=capacity=year=city=metro=deposit=kmIncluded=kasko=0
    osago=fuel=volume=ratingOwner=loyaltyOwner=carsOwner=avgPriceOwner=noDeposit=delivery=returnAbility=kmUnlimited=childSeat=roofRack=0
    values_char = [gearbox, engine, body, drive, ecologyEU, ecologyRF, rudder, doors, run, salon, capacity, year, city, metro, 
                   deposit, kmIncluded, kasko, osago, fuel, volume]
    names_char = ['двигатель', 'КПП', 'Кузов', 'привод', 'Экологический стандарт ЕС', 'Экологический стандарт РФ', 'руль', 'дверей',
                  'Пробег', 'салон', 'мест', 'Год', 'Город', 'Метро', 'Депозит', 'Включенные КМ', 'Каско', 'Осаго', 'Марка топлива',
                 'объем']
    values_owner = [ratingOwner, loyaltyOwner, carsOwner, avgPriceOwner]
    names_owner = ['средняя оценка арендаторов', 'арендаторов рекомендуют', 'автомобилей в каталоге', 
                    'средняя стоимость автомобилей']
    values_service = [noDeposit, delivery, returnAbility, kmUnlimited, childSeat, roofRack]
    names_service = ['Без депозита', 'Доставка', 'Возврат', 'Без ограничения пробега', 'Детское кресло', 'Багажник на крышу']
    
    #создаем переменные характеристик автомобиля и арендного предложения
    for j in range(len(names_char)):
        try:
            values_char[j] = char_values[char_names.index(names_char[j])]
        except:
            values_char[j] = None
    gearbox, engine, body, drive, ecologyEU, ecologyRF, rudder, doors, run, salon, capacity, year, city, metro, deposit, kmIncluded, kasko, osago, fuel, volume = values_char
    
    #создаем переменные характеристики владельца
    for k in range(len(names_owner)):
        try:
            values_owner[k] = char_owner[char_owner_names.index(names_owner[k])]
        except:
            values_owner[k] = None
    ratingOwner, loyaltyOwner, carsOwner, avgPriceOwner = values_owner
    
    #создаем переменные дополнительных услуг
    for n in range(len(names_service)):
        try:
            services.index(names_service[n])
            values_service[n] = 1
        except:
            values_service[n] = 0
    noDeposit, delivery, returnAbility, kmUnlimited, childSeat, roofRack = values_service
    
    #описание автомобиля
    description = treeCar.find('div', {'class': 'about-car'})
    if description != None:
        description = treeCar.find('div', {'class': 'about-car'}).text

    
    #Запись полученных переменных
    data.append({'URL': urlCar,
             'Price': price_day,
             'Sale1': sale1,
             'Sale2': sale2,
             'Sale3': sale3,
             'Sale4': sale4,
             'Brand': brand,
             'Model': model,
             'Owner': owner,
             'PhotoOwner': photo_owner,
             'RatingOwner': ratingOwner,
             'LoyaltyOwner': loyaltyOwner,
             'CarsOwner': carsOwner,
             'AvgPriceOwner': avgPriceOwner,
             'PhotosCar': photos_car,
             'Rating': rating,
             'N_feedback': n_feedback,
             'Gearbox': gearbox,
             'Engine': engine,
             'Fuel': fuel,
             'Volume': volume,
             'Body': body,
             'Drive': drive,
             'EcologyEU': ecologyEU,
             'EcologyRF': ecologyRF,
             'Rudder': rudder,
             'Doors': doors,
             'Run': run,
             'Capacity': capacity,
             'Class': car_class,
             'Year': year,
             'City': city,
             'Metro': metro,
             'Deposit': deposit,
             'KmIncluded': kmIncluded,
             'Kasko': kasko,
             'Osago': osago,
             'NoDeposit': noDeposit,
             'Delivery': delivery,
             'ReturnAbility': returnAbility,
             'KmUnlimited': kmUnlimited,
             'ChildSeat': childSeat,
             'RoofRack': roofRack,
             'Description': description,
             'CharacterCar': char_names,
             'CharacterOwner': char_owner_names,
             'Services': services})

In [10]:
data = []

In [42]:
url='https://rentride.ru/arendovat/habarovsk/' #сюда подставить веб-страницу на аренду в определенном городе
response = requests.get(url, headers={'User-Agent': UserAgent().chrome})
tree = BeautifulSoup(response.content, 'html.parser')
cars = tree.find_all('a', {'class': 'car-card-vertical'})
for car in tqdm(cars):
    link = car.get('href')
    urlCar = 'https://rentride.ru' + link
    responseCar = requests.get(urlCar, headers={'User-Agent': UserAgent().chrome})
    treeCar = BeautifulSoup(responseCar.content, 'html.parser') 
    get_car_data(treeCar, car)

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  2.32it/s]


In [41]:
for p in tqdm(range(1,3)): #в url подставить веб-страницу на аренду в определенном городе, где несколько страниц
    url='https://rentride.ru/arendovat/sochi/?page='+str(p)
    response = requests.get(url, headers={'User-Agent': UserAgent().chrome})
    tree = BeautifulSoup(response.content, 'html.parser')
    cars = tree.find_all('a', {'class': 'car-card-vertical'})
    for car in tqdm(cars):
        link = car.get('href')
        urlCar = 'https://rentride.ru' + link
        responseCar = requests.get(urlCar, headers={'User-Agent': UserAgent().chrome})
        treeCar = BeautifulSoup(responseCar.content, 'html.parser') 
        get_car_data(treeCar, car)

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:13<00:00,  6.95s/it]


In [43]:
df = pd.DataFrame(data)
df

,URL,Price,Sale1,Sale2,Sale3,Sale4,Brand,Model,Owner,PhotoOwner,RatingOwner,LoyaltyOwner,CarsOwner,AvgPriceOwner,PhotosCar,Rating,N_feedback,Gearbox,Engine,Fuel,Volume,Body,Drive,EcologyEU,EcologyRF,Rudder,Doors,Run,Capacity,Class,Year,City,Metro,Deposit,KmIncluded,Kasko,Osago,NoDeposit,Delivery,ReturnAbility,KmUnlimited,ChildSeat,RoofRack,Description,CharacterCar,CharacterOwner,Services
0,https://rentride.ru/cars/503489/,2400,От 4 суток -5% 2850 ₽ / сутки,От 7 суток -10% 2700 ₽ / сутки,От 14 суток -15% 2550 ₽ / сутки,От 21 суток -20% 2400 ₽ / сутки,Citroen,Jumper,Сергей Тарадай,None,None,None,4+,3000 ₽,5,0,Ещё нет отзывов,Дизель,Механическая,None,2.2,Микроавтобус,Передний,Euro-5,пятый,Руль слева,5,> 200тыс.км,8,Эконом,2014,None,None,None,None,Без страховки,Впишу в полис,1,0,0,0,0,0,Грузопассажирский,"[привод, Кузов, двигатель, КПП, tariff, Эколог...","[автомобилей в каталоге, средняя стоимость авт...","[Без депозита, Сервис, Помощь]"
1,https://rentride.ru/cars/502717/,2000,От 4 суток -5% 2375 ₽ / сутки,От 7 суток -10% 2250 ₽ / сутки,От 14 суток -15% 2125 ₽ / сутки,От 21 суток -20% 2000 ₽ / сутки,Volkswagen,Polo,Мария Иванова,https://storage.rentride.ru/uploads/users/5666...,None,None,3+,2983 ₽,3,0,Ещё нет отзывов,Бензин,Автоматическая,None,None,Седан,None,Euro-5,пятый,Руль слева,5,< 100тыс.км,5,Эконом,2019,Анапа,None,5000,300,Мультидрайв,Без страховки,0,1,1,1,1,0,None,"[двигатель, КПП, Кузов, tariff, Экологический ...","[автомобилей в каталоге, средняя стоимость авт...","[Детское кресло, Без ограничения пробега, Возв..."
2,https://rentride.ru/cars/502718/,2800,От 4 суток -5% 3325 ₽ / сутки,От 7 суток -10% 3150 ₽ / сутки,От 14 суток -15% 2975 ₽ / сутки,От 21 суток -20% 2800 ₽ / сутки,Volkswagen,Tiguan,Мария Иванова,https://storage.rentride.ru/uploads/users/5666...,None,None,3+,2983 ₽,2,0,Ещё нет отзывов,Бензин,Автоматическая,None,None,Внедорожник,None,Euro-5,пятый,Руль слева,5,< 200тыс.км,5,Эконом,2011,Анапа,None,5000,None,Без страховки,Мультидрайв,0,1,1,1,1,0,None,"[двигатель, КПП, Кузов, tariff, Экологический ...","[автомобилей в каталоге, средняя стоимость авт...","[Детское кресло, Без ограничения пробега, Возв..."
3,https://rentride.ru/cars/500938/,3500,От 4 суток -7% 4625 ₽ / сутки,От 7 суток -15% 4250 ₽ / сутки,От 14 суток -22% 3875 ₽ / сутки,От 21 суток -30% 3500 ₽ / сутки,Mercedes-Benz,Vito,Андрей Хозиков,https://storage.rentride.ru/uploads/users/5484...,None,None,3+,4250 ₽,12,0,Ещё нет отзывов,Дизель,Механическая,None,2.1,Минивэн,Задний,None,None,Руль слева,None,> 200тыс.км,7,Эконом,2007,Анапа,None,15000,300,Без страховки,Впишу в полис,0,0,0,0,0,0,Авто в идеальном состоянии,"[двигатель, КПП, привод, Кузов, класс, мест, о...","[автомобилей в каталоге, средняя стоимость авт...","[Сервис, Помощь]"
4,https://rentride.ru/cars/504014/,1600,От 4 суток -5% 1900 ₽ / сутки,От 7 суток -10% 1800 ₽ / сутки,От 14 суток -15% 1700 ₽ / сутки,От 21 суток -20% 1600 ₽ / сутки,Infiniti,QX60,Егор Федотов,https://storage.rentride.ru/uploads/users/5768...,None,None,None,None,12,0,Ещё нет отзывов,Бензин,Вариатор,None,3.5,Кроссовер,Полный,None,None,Руль слева,4,< 100тыс.км,7,Эконом,2014,Бронницы,None,45000,None,Без страховки,Без страховки,1,0,0,0,0,0,Отличный автомобиль. Вместимость - 7 мест. Ког...,"[привод, Кузов, двигатель, КПП, мест, дверей, ...",[],"[Без депозита, Сервис, Помощь]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
651,https://rentride.ru/cars/185186/,3800,От 4 суток -6% 4700 ₽ / сутки,От 7 суток -12% 4400 ₽ / сутки,От 14 суток -18% 4100 ₽ / сутки,От 21 суток -24% 3800 ₽ / сутки,Hyundai,H1\Grand Starex,Елена Шелестова,https://storage.rentride.ru/uploads/users/1011...,None,None,14+,2877 ₽,1,0,Ещё нет отзывов,Дизель,Автоматическая,petrol,2.5,Минивэн,Задний,Euro-5,пятый,Руль слева,4,< 100тыс.км,9,Эконом,2012,Сочи,Котельники,10000,250,Без страховки,Мультидрайв,0,1,1,0,0,0,"Прокат, аренда 

In [49]:
df.to_csv('RentRide.csv', encoding = 'utf-8-sig')

In [48]:
df.to_excel("RentRide.xlsx", encoding = 'utf-8')

Мотоциклы:

- Махачкала
- Москва, стр. 13
- Химки